# AutoEncoders

## Downloading the dataset

### ML-100K

In [ ]:
!wget "http://files.grouplens.org/datasets/movielens/ml-100k.zip"
!unzip ml-100k.zip
!ls

--2022-02-16 10:14:33--  http://files.grouplens.org/datasets/movielens/ml-100k.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4924029 (4.7M) [application/zip]
Saving to: ‘ml-100k.zip’

ml-100k.zip         100%[===================>]   4.70M  11.2MB/s    in 0.4s    

2022-02-16 10:14:34 (11.2 MB/s) - ‘ml-100k.zip’ saved [4924029/4924029]

Archive:  ml-100k.zip
   creating: ml-100k/
  inflating: ml-100k/allbut.pl       
  inflating: ml-100k/mku.sh          
  inflating: ml-100k/README          
  inflating: ml-100k/u.data          
  inflating: ml-100k/u.genre         
  inflating: ml-100k/u.info          
  inflating: ml-100k/u.item          
  inflating: ml-100k/u.occupation    
  inflating: ml-100k/u.user          
  inflating: ml-100k/u1.base         
  inflating: ml-100k/u1.test         
  inflating: ml-100k/u2.base

### ML-1M

In [ ]:
!wget "http://files.grouplens.org/datasets/movielens/ml-1m.zip"
!unzip ml-1m.zip
!ls

--2022-02-16 10:14:34--  http://files.grouplens.org/datasets/movielens/ml-1m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5917549 (5.6M) [application/zip]
Saving to: ‘ml-1m.zip’

ml-1m.zip           100%[===================>]   5.64M  13.3MB/s    in 0.4s    

2022-02-16 10:14:35 (13.3 MB/s) - ‘ml-1m.zip’ saved [5917549/5917549]

Archive:  ml-1m.zip
   creating: ml-1m/
  inflating: ml-1m/movies.dat        
  inflating: ml-1m/ratings.dat       
  inflating: ml-1m/README            
  inflating: ml-1m/users.dat         
drive  ml-100k	ml-100k.zip  ml-1m  ml-1m.zip  sample_data


### Importing the libraries

In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

### Importing the dataset

In [ ]:
# We won't be using this dataset.
movies = pd.read_csv('ml-1m/movies.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
users = pd.read_csv('ml-1m/users.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
ratings = pd.read_csv('ml-1m/ratings.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')

###  Preparing the training set and the test set

In [ ]:
training_set = pd.read_csv('ml-100k/u1.base', delimiter = '\t')
training_set = np.array(training_set, dtype = 'int')
test_set = pd.read_csv('ml-100k/u1.test', delimiter = '\t')
test_set = np.array(test_set, dtype = 'int')

### Getting the number of users and movies

In [ ]:
nb_users = int(max(max(training_set[:, 0], ), max(test_set[:, 0])))
nb_movies = int(max(max(training_set[:, 1], ), max(test_set[:, 1])))

### Converting the data into an array with users in lines and movies in columns

In [ ]:
def convert(data):
  new_data = []
  for id_users in range(1, nb_users + 1):
    id_movies = data[:, 1] [data[:, 0] == id_users]
    id_ratings = data[:, 2] [data[:, 0] == id_users]
    ratings = np.zeros(nb_movies)
    ratings[id_movies - 1] = id_ratings
    new_data.append(list(ratings))
  return new_data
training_set = convert(training_set)
test_set = convert(test_set)

### Converting the data into Torch tensors

In [ ]:
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

### Creating the architecture of the Neural Network

In [9]:
class SAE(nn.Module):
    #this is a Stacked AutoEncoder
    def __init__(self, ):
        super(SAE, self).__init__()   #super lets us use methods and classes of nn module
        self.fc1 = nn.Linear(nb_movies, 20)    
        self.fc2 = nn.Linear(20, 10)           
        self.fc3 = nn.Linear(10, 20)
        self.fc4 = nn.Linear(20, nb_movies)
        self.activation = nn.Sigmoid()
    def forward(self, x):     
        #not only will help in encoding and decoding but also will apply to different activation functions inside the fc
        #this function also returns a vector of predicted ratings that we'll compare to the real ratings (input vector) 
        x = self.activation(self.fc1(x))  #returns 1st encoded vector
        x = self.activation(self.fc2(x))  #returns 2nd encoded vector
        x = self.activation(self.fc3(x))  #returns decoded vector
        x = self.fc4(x)                   #last part of decoding so we use the last layer without activation function
        return x      #vector of predicted ratings
sae = SAE()
criterion = nn.MSELoss()
optimizer = optim.RMSprop(sae.parameters(), lr = 0.01, weight_decay = 0.5) #lr: learning rate, weight_decay helps in convergence

### Training the SAE

In [11]:
nb_epoch = 200
for epoch in range(1, nb_epoch + 1):
  train_loss = 0
  s = 0.
  for id_user in range(nb_users):
    input = Variable(training_set[id_user]).unsqueeze(0)
    target = input.clone()
    if torch.sum(target.data > 0) > 0:
      output = sae(input)
      target.require_grad = False
      output[target == 0] = 0
      loss = criterion(output, target)
      mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
      loss.backward()
      train_loss += np.sqrt(loss.data*mean_corrector)
      s += 1.
      optimizer.step()
  print('epoch: '+str(epoch)+' loss: '+ str(train_loss/s))

epoch: 1 loss: tensor(1.0184)
epoch: 2 loss: tensor(1.0179)
epoch: 3 loss: tensor(1.0174)
epoch: 4 loss: tensor(1.0172)
epoch: 5 loss: tensor(1.0169)
epoch: 6 loss: tensor(1.0165)
epoch: 7 loss: tensor(1.0166)
epoch: 8 loss: tensor(1.0160)
epoch: 9 loss: tensor(1.0162)
epoch: 10 loss: tensor(1.0160)
epoch: 11 loss: tensor(1.0159)
epoch: 12 loss: tensor(1.0157)
epoch: 13 loss: tensor(1.0159)
epoch: 14 loss: tensor(1.0156)
epoch: 15 loss: tensor(1.0156)
epoch: 16 loss: tensor(1.0151)
epoch: 17 loss: tensor(1.0152)
epoch: 18 loss: tensor(1.0126)
epoch: 19 loss: tensor(1.0114)
epoch: 20 loss: tensor(1.0103)
epoch: 21 loss: tensor(1.0079)
epoch: 22 loss: tensor(1.0072)
epoch: 23 loss: tensor(1.0040)
epoch: 24 loss: tensor(1.0023)
epoch: 25 loss: tensor(1.0004)
epoch: 26 loss: tensor(0.9983)
epoch: 27 loss: tensor(0.9957)
epoch: 28 loss: tensor(0.9947)
epoch: 29 loss: tensor(0.9894)
epoch: 30 loss: tensor(0.9892)
epoch: 31 loss: tensor(0.9855)
epoch: 32 loss: tensor(0.9849)
epoch: 33 loss: t

### Testing the SAE

In [12]:
test_loss = 0
s = 0.
for id_user in range(nb_users):
  input = Variable(training_set[id_user]).unsqueeze(0)
  target = Variable(test_set[id_user]).unsqueeze(0)
  if torch.sum(target.data > 0) > 0:
    output = sae(input)
    target.require_grad = False
    output[target == 0] = 0
    loss = criterion(output, target)
    mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
    test_loss += np.sqrt(loss.data*mean_corrector)
    s += 1.
print('test loss: '+str(test_loss/s))

test loss: tensor(0.9755)
